In [1]:
from glob import glob
import pyarrow.parquet as pq
import pandas as pd
ROOT_DIR = "/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M"
all_files = glob(ROOT_DIR+"/*")
print(len(all_files), all_files)
from tqdm import tqdm
import json
from transformers import AutoTokenizer
model_path = "/DATA/disk2/yuhang/.cache/modelscope/models/Qwen/Qwen2___5-0___5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
import numpy as np

75 ['/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00050-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00007-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00030-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00014-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00062-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00043-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00065-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00066-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M/train-00061-of-00075.parquet', '/DATA/disk2/yuhang/.cache/modelscope/datasets/BAAI/Infinity

/home/chenyuhang/anaconda3/envs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 转为llama factory需要的格式

In [2]:

def to_alpaca_data(str_json_data):
    return {
                "instruction": str_json_data[-2]["value"],
                "input": "",
                "output": str_json_data[-1]["value"]
            }

def to_share_gpt_data(str_json_data):
    conversations = str_json_data.tolist()
    text = ""
    for item in conversations:
        text += item["value"]
    return {
        "conversations": conversations
    }, text

def process_one_data(file_dir):
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    # Index(['id', 'conversations', 'label', 'langdetect', 'source'], dtype='object')
    # print(df.columns)
    # 可以过滤=======================
    # need_idx = df["langdetect"] == "zh-cn"
    # df = df.loc[need_idx, :]
    #===============================
    str_json_datas = df["conversations"]
    language_datas = df["langdetect"]
    muti_turn_counter = 0
    odd_cov_counter = 0
    encounter = 0
    a_file_datas = []
    text_list = []
    for idx, str_json_data in  enumerate(str_json_datas):
        L = len(str_json_data)
        #! 过滤中文
        if language_datas.iloc[idx] != "zh-cn":
            encounter += 1
            continue
        #! 过滤多轮对话
        if L > 2:
            muti_turn_counter += 1
        if L % 2 == 1:
            odd_cov_counter += 1
        # a_file_datas.append(to_alpaca_data(str_json_data)) 
        conversation, text= to_share_gpt_data(str_json_data)
        text_list.append(text)
        a_file_datas.append(conversation)
    
    input_ids = tokenizer(text_list)['input_ids']
    token_num_list = [len(x) for x in input_ids]
    print(f"all data:{len(str_json_datas)} muti_turn_counter:{muti_turn_counter} odd_cov_counter:{odd_cov_counter} encounter:{encounter}")
    return a_file_datas,token_num_list

all_datas = []
token_nums = []
for file_dir in tqdm(all_files):
    datas,token_num_list = process_one_data(file_dir) 
    all_datas.extend(datas) 
    token_nums.extend(token_num_list)
print(len(all_datas))
print(len(token_nums))

  1%|▏         | 1/75 [00:02<02:28,  2.00s/it]

all data:100000 muti_turn_counter:2421 odd_cov_counter:0 encounter:89869


  3%|▎         | 2/75 [00:04<02:31,  2.07s/it]

all data:100000 muti_turn_counter:2444 odd_cov_counter:0 encounter:89987


  4%|▍         | 3/75 [00:06<02:29,  2.07s/it]

...

all data:100000 muti_turn_counter:2431 odd_cov_counter:0 encounter:89981


 99%|█████████▊| 74/75 [02:33<00:02,  2.07s/it]

all data:100000 muti_turn_counter:2498 odd_cov_counter:0 encounter:89905


100%|██████████| 75/75 [02:35<00:00,  2.07s/it]

all data:100000 muti_turn_counter:2421 odd_cov_counter:0 encounter:90054
751313
751313


# 统计分位数，过滤过长的样本

In [4]:
import numpy as np
np.percentile(token_nums,[50, 60, 70, 80, 90,95,99])
print(np.percentile(token_nums,[50, 60, 70, 80, 90,95, 99]))
filtered_long_data = []
filtered_token_num = []
for i in range(len(all_datas)):
    if token_nums[i] < 768:
        filtered_long_data.append(all_datas[i])
        filtered_token_num.append(token_nums[i])
print(np.percentile(filtered_token_num,[50, 60, 70, 80, 90,95, 99]))
print(len(filtered_token_num))

[ 252.  281.  317.  368.  474.  666. 1692.]
[247. 274. 306. 349. 423. 504. 679.]
722236


In [6]:
import os
import os
file_path = '/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input/BAAI_Infinity-Instruct_7M.jsonl'

directory = os.path.dirname(file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"目录{directory} 不存在，已创建")
else:
    print(f"目录{directory} 已存在")

with open(file_path, 'w', encoding="utf-8") as f:
    for item in filtered_long_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

目录/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input 已存在


In [7]:
# 随机保存7w个数据
import random
random_data = random.sample(all_datas, 100000)
print(type(all_datas[0]))
import json
print(all_datas[0])
json_dir = "/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input/BAAI_Infinity-Instruct_7M_10W.jsonl"
with open(json_dir, "w") as file:
    json.dump(all_datas, file)

<class 'dict'>
{'conversations': [{'from': 'human', 'value': "请从下面的文本中抽取出事件的参数,文本内容以text:<>给出.并按照列表格式输出，列表中每个元素为一个事件对应的字典，例如{'role':'事件','参数':'事件参数'}text:<受损股民可至新浪股民维权平台登记该公司维权：http://wq.finance.sina.com.cn/\n微博关注@新浪证券、微信关注新浪券商基金、百度搜索新浪股民维权、访问新浪财经客户端、新浪财经首页都能找到我们！\n新浪财经讯 暂停上市的乐视网(2.690,-0.05,-1.82%)（维权）（300104），2019年10月14日《2019年前三季度业绩预告》发布了“恐怖”的数据：归属于上市公司股东的净利润亏损1，019，710.46万元—1，020，210.46万元，并提醒投资者，公司存在 2019 年全年经审计后归属于上市公司股东的净资产为负的风险。最近又传来贾跃亭在美国申请个人破产重组的消息。（乐视网维权入口）\n这些消息无疑给受损的股民提起索赔蒙上了阴影。\n新浪财经上市公司研究院的专业咨询支持单位北京德恒（宁波）律师事务所的高级合伙人张志旺律师对此进行专业解读。\n张志旺律师介绍，2019 年 5月 13 日起乐视网因为资不抵债（2018年度归母公司净资产为负）而被暂停上市。实际上乐视网在年初，是想避免这结果的发生。2019年1月30日晚披露2018年度业绩预告，称预计亏损6.08亿元至6.13亿元。因为2017年净资产为6.6亿，大于亏损额6.13亿，则2018年净资产避免为负数，站在生死线之上。但乐视网的努力遭到深交所的关注，原因是乐视网认为所持乐融致新（新乐视智家）预计确认投资收益20.99亿元。深交所指出：核实说明是否通过评估值选取及重估收益确认，调节公司净利润、净资产等业绩指标以规避退市风险>"}, {'from': 'gpt', 'value': "[{'role': '披露时间', 'argument': '2019年10月14日'}, {'role': '净亏损', 'argument': '1，019，710.46万元—1，020，210.46万元'}, {'role': '财报周期', 'argument': '2019年前三

In [8]:
import json
with open("/DATA/disk2/yuhang/.cache/steel_dataset/sft_data/llamafactory_input/BAAI_Infinity-Instruct_7M_10W.jsonl", "r") as file:
    read_data = json.load(file)